# Compas First Training

In [43]:
import pandas as pd 
import numpy as np
from sklearn.linear_model import LogisticRegression 
from sklearn.svm import SVC 
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, accuracy_score

In [44]:
def dp_diff(y_priv, y_unpriv):
    
    dp_priv = sum(y_priv)/len(y_priv)
    dp_unpriv = sum(y_unpriv)/len(y_unpriv)
    return dp_priv-dp_unpriv


def disparate_impact(y_priv, y_unpriv):
    dp_priv = sum(y_priv)/len(y_priv)
    dp_unpriv = sum(y_unpriv)/len(y_unpriv)
    return dp_unpriv/dp_priv

In [45]:
DATASET = "compas-scores-two-years.csv"


In [46]:
df = pd.read_csv(DATASET)

### In order to create some classifiers, the dataset had to be processed. Among the large number of columns, we chose a few, and the choice was made mostly based on what we considered essential for our study. For instance, we chose: 
1. Race, sex, age category, both sensitive attributes and important aspects of each individual
2. Juvenile encounters with the law, we have three columns of this kind in the dataset.
3. The number of prior offenses.
4. The time spent in jail, in days, computed as the difference between the date of entry and release from jail.

### We focused on the sensitive attributes, and the attributes that are connected to encounters with police/law. 

In [47]:
def process_compas_binary_sensitive_attributes(df):

    df = df.dropna(subset=["days_b_screening_arrest", "race"]) # dropping missing vals
    print(df.shape)

    df = df[ (df.days_b_screening_arrest <= 30) &
    (df.days_b_screening_arrest >= -30) &
    (df.is_recid != -1) & (df.c_charge_degree != 'O') & (df.score_text != 'N/A') ]

    df.reset_index(inplace=True, drop=True) # renumber the rows from 0 again
    print(df.shape)
    print(df.columns)

    relevant_attributes = ['sex',  'age_cat', 'race' ,'juv_fel_count', 'juv_misd_count', 
        'juv_other_count', 'priors_count',  'c_jail_out', 'c_jail_in', 'two_year_recid']

    df_relev = df[relevant_attributes]
    df_relev['length_of_stay'] = (pd.to_datetime(df_relev['c_jail_out']) -
                                              pd.to_datetime(df_relev['c_jail_in'])).apply(
                                              lambda x: x.days)
    df_relev['sex'] = df_relev['sex'].map( {'Male':1, 'Female':0} )

    df_relev['race'] = df_relev['race'].map({'Caucasian':1, 'African-American': 0, 'Asian': -1, 'Native American': -1, 'Other': -1, 'Hispanic': -1})


    df_relev = df_relev[ (df_relev.race != -1) ]


    df_relev = pd.concat([df_relev, pd.get_dummies(df_relev['age_cat'], prefix='age_cat')], axis=1)
    df_relev = df_relev.drop(['age_cat', 'c_jail_out', 'c_jail_in'], axis=1)

    df_relev = df_relev.dropna(subset=['race'])
    df_relev['not_recidivist'] = 1- df_relev['two_year_recid']
    df_relev = df_relev.drop(['two_year_recid'], axis=1)

    df_relev.to_csv('compas_processed.csv', index=False)
    print(df_relev.head())


process_compas_binary_sensitive_attributes(df)

(6907, 53)
(6172, 53)
Index(['id', 'name', 'first', 'last', 'compas_screening_date', 'sex', 'dob',
       'age', 'age_cat', 'race', 'juv_fel_count', 'decile_score',
       'juv_misd_count', 'juv_other_count', 'priors_count',
       'days_b_screening_arrest', 'c_jail_in', 'c_jail_out', 'c_case_number',
       'c_offense_date', 'c_arrest_date', 'c_days_from_compas',
       'c_charge_degree', 'c_charge_desc', 'is_recid', 'r_case_number',
       'r_charge_degree', 'r_days_from_arrest', 'r_offense_date',
       'r_charge_desc', 'r_jail_in', 'r_jail_out', 'violent_recid',
       'is_violent_recid', 'vr_case_number', 'vr_charge_degree',
       'vr_offense_date', 'vr_charge_desc', 'type_of_assessment',
       'decile_score.1', 'score_text', 'screening_date',
       'v_type_of_assessment', 'v_decile_score', 'v_score_text',
       'v_screening_date', 'in_custody', 'out_custody', 'priors_count.1',
       'start', 'end', 'event', 'two_year_recid'],
      dtype='object')
   sex  race  juv_fel_count

/tmp/ipykernel_5685/1430328400.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_relev['length_of_stay'] = (pd.to_datetime(df_relev['c_jail_out']) -
/tmp/ipykernel_5685/1430328400.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_relev['sex'] = df_relev['sex'].map( {'Male':1, 'Female':0} )
/tmp/ipykernel_5685/1430328400.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

In [48]:
data = pd.read_csv('compas_processed.csv')
data.shape
y = data['not_recidivist'].to_numpy()
X = data.drop(['not_recidivist'], axis=1).to_numpy()
print(X.shape)
print(y.shape)


(5278, 10)
(5278,)


In [49]:
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)




In [50]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


(3694, 10)
(1584, 10)
(3694,)
(1584,)


### We are training three different classifiers, and we are going to compare their results. 

In [51]:
logistic_regression = LogisticRegression(penalty="l2", solver='lbfgs', C=0.01)
svm = SVC(kernel='linear', C=0.1)
mlp = MLPClassifier(hidden_layer_sizes =(12,))


In [52]:
logistic_regression.fit(X_train, y_train)
svm.fit(X_train, y_train)
mlp.fit(X_train, y_train)


MLPClassifier(hidden_layer_sizes=(12,))

In [53]:
y_pred_log = logistic_regression.predict(X_test)
y_pred_svm = svm.predict(X_test)
y_pred_mlp = mlp.predict(X_test)


In [54]:
def evaluate(name, y_true, y_pred):
    print("For " + name + " classifier, we obtained: ")
    print("Accuracy: ", accuracy_score(y_true, y_pred) )
    print("Precision: ", precision_score(y_true, y_pred) )
    print("Recall: ", recall_score(y_true, y_pred) )


evaluate("logistic regression", y_test, y_pred_log)
evaluate("svm", y_test, y_pred_svm)
evaluate("mlp", y_test, y_pred_mlp)


For logistic regression classifier, we obtained: 
Accuracy:  0.6559343434343434
Precision:  0.6551020408163265
Recall:  0.7561837455830389
For svm classifier, we obtained: 
Accuracy:  0.6464646464646465
Precision:  0.6529100529100529
Recall:  0.7267373380447585
For mlp classifier, we obtained: 
Accuracy:  0.6584595959595959
Precision:  0.668859649122807
Recall:  0.71849234393404


In [55]:

mask = (X_test[:, 1] > 0)
X_test_white = X_test[mask, :]
y_test_white = y_test[mask]

mask = (X_test[:, 1] < 0)
X_test_black = X_test[mask, :]
y_test_black = y_test[mask]

print(X_test_white.shape)
print(X_test_black.shape)
print(y_test_black.shape)


(618, 10)
(966, 10)
(966,)


In [56]:
y_pred_black = logistic_regression.predict(X_test_black)
y_pred_white = logistic_regression.predict(X_test_white)

evaluate('log reg black', y_test_black, y_pred_black)
evaluate('log reg white', y_test_white, y_pred_white)

print("-----------------------------------------------")

y_pred_black = svm.predict(X_test_black)
y_pred_white = svm.predict(X_test_white)

evaluate('svm black', y_test_black, y_pred_black)
evaluate('svm white', y_test_white, y_pred_white)
print("-----------------------------------------------")

y_pred_black = mlp.predict(X_test_black)
y_pred_white = mlp.predict(X_test_white)

evaluate('mlp black', y_test_black, y_pred_black)
evaluate('mlp white', y_test_white, y_pred_white)



For log reg black classifier, we obtained: 
Accuracy:  0.6656314699792961
Precision:  0.6630669546436285
Recall:  0.6476793248945147
For log reg white classifier, we obtained: 
Accuracy:  0.6407766990291263
Precision:  0.6479690522243714
Recall:  0.8933333333333333
-----------------------------------------------
For svm black classifier, we obtained: 
Accuracy:  0.65527950310559
Precision:  0.6453608247422681
Recall:  0.6603375527426161
For svm white classifier, we obtained: 
Accuracy:  0.6326860841423948
Precision:  0.6608695652173913
Recall:  0.8106666666666666
-----------------------------------------------
For mlp black classifier, we obtained: 
Accuracy:  0.6614906832298136
Precision:  0.6666666666666666
Recall:  0.620253164556962
For mlp white classifier, we obtained: 
Accuracy:  0.6537216828478964
Precision:  0.6709129511677282
Recall:  0.8426666666666667


In [57]:
di = disparate_impact(y_pred_white, y_pred_black)
di

0.5990030462475768

### A very naive bias mittigation strategy is to simply drop the sensitive attribute alltogether. We chose to drop race and sex. We are going to train this here.

In [58]:
# data = pd.read_csv('compas_processed.csv')
# data.shape
# y = data['not_recidivist'].to_numpy()
# X = data.drop(['not_recidivist', 'race', 'sex'], axis=1).to_numpy()
# print(X.shape)
# print(y.shape)

In [59]:
# scaler = StandardScaler()
# scaler.fit(X)
# X = scaler.transform(X)

In [60]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# print(X_train.shape)
# print(X_test.shape)
# print(y_train.shape)
# print(y_test.shape)


# drop the race and the sex from the dataset, they are the first two columns
X_train = np.delete(X_train, 0, 1)
X_train = np.delete(X_train, 0, 1)


X_test_prev = X_test
X_test= np.delete(X_test, 0, 1)
X_test = np.delete(X_test, 0, 1)


In [61]:
logistic_regression = LogisticRegression(penalty="l2", solver='lbfgs', C=0.01)
svm = SVC(kernel='linear', C=0.1)
mlp = MLPClassifier(hidden_layer_sizes =(12,))

In [62]:
logistic_regression.fit(X_train, y_train)
svm.fit(X_train, y_train)
mlp.fit(X_train, y_train)

MLPClassifier(hidden_layer_sizes=(12,))

In [63]:
y_pred_log = logistic_regression.predict(X_test)
y_pred_svm = svm.predict(X_test)
y_pred_mlp = mlp.predict(X_test)

In [64]:
evaluate("logistic regression", y_test, y_pred_log)
evaluate("svm", y_test, y_pred_svm)
evaluate("mlp", y_test, y_pred_mlp)

For logistic regression classifier, we obtained: 
Accuracy:  0.6565656565656566
Precision:  0.6496565260058881
Recall:  0.7797408716136631
For svm classifier, we obtained: 
Accuracy:  0.6458333333333334
Precision:  0.6518987341772152
Recall:  0.7279151943462897
For mlp classifier, we obtained: 
Accuracy:  0.6515151515151515
Precision:  0.650761421319797
Recall:  0.7550058892815077


In [65]:
mask = (X_test_prev[:, 1] > 0)
X_test_white = X_test[mask, :]
y_test_white = y_test[mask]

mask = (X_test_prev[:, 1] < 0)
X_test_black = X_test[mask, :]
y_test_black = y_test[mask]

print(X_test_white.shape)
print(X_test_black.shape)
print(y_test_black.shape)

(618, 8)
(966, 8)
(966,)


In [66]:
y_pred_black = logistic_regression.predict(X_test_black)
y_pred_white = logistic_regression.predict(X_test_white)

evaluate('log reg black', y_test_black, y_pred_black)
evaluate('log reg white', y_test_white, y_pred_white)

print("-----------------------------------------------")

y_pred_black = svm.predict(X_test_black)
y_pred_white = svm.predict(X_test_white)

evaluate('svm black', y_test_black, y_pred_black)
evaluate('svm white', y_test_white, y_pred_white)
print("-----------------------------------------------")

y_pred_black = mlp.predict(X_test_black)
y_pred_white = mlp.predict(X_test_white)

evaluate('mlp black', y_test_black, y_pred_black)
evaluate('mlp white', y_test_white, y_pred_white)

For log reg black classifier, we obtained: 
Accuracy:  0.6666666666666666
Precision:  0.6407407407407407
Recall:  0.729957805907173
For log reg white classifier, we obtained: 
Accuracy:  0.6407766990291263
Precision:  0.6597077244258872
Recall:  0.8426666666666667
-----------------------------------------------
For svm black classifier, we obtained: 
Accuracy:  0.6563146997929606
Precision:  0.6454918032786885
Recall:  0.6645569620253164
For svm white classifier, we obtained: 
Accuracy:  0.6294498381877023
Precision:  0.658695652173913
Recall:  0.808
-----------------------------------------------
For mlp black classifier, we obtained: 
Accuracy:  0.6532091097308489
Precision:  0.6323809523809524
Recall:  0.70042194092827
For mlp white classifier, we obtained: 
Accuracy:  0.6488673139158576
Precision:  0.6717391304347826
Recall:  0.824


In [67]:
di = disparate_impact(y_pred_white, y_pred_black)
di

0.7301512287334593